# Blockage modelling

This example shows how to use the Rankine-Half-Body model for modelling the blockage effect. We need the following imports:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import foxes
import foxes.variables as FV
import foxes.constants as FC

First, we setup the model book and the wind farm, here we pick 20 x 10 turbines on a regular grid:

In [ ]:
mbook = foxes.ModelBook()

farm = foxes.WindFarm()
foxes.input.farm_layout.add_grid(
    farm=farm,
    xy_base=[0.0, 0.0],
    step_vectors=[[800.0, 0.0], [0., 400.0]],
    steps=[20, 10],
    turbine_models=["DTU10MW", "kTI_05"],
    verbosity=0
)

ax = foxes.output.FarmLayoutOutput(farm).get_figure(figsize=(8, 4))
plt.show()

As wind conditions, we choose a single wind direction state from 270 degrees:

In [ ]:
# create ambient wind conditions, a single uniform state:
states = foxes.input.states.SingleStateStates(ws=8.0, wd=270.0, ti=0.04, rho=1.225)

For induction models, the `Iterative` algorithm should be invoked:

In [ ]:
algo = foxes.algorithms.Iterative(
    mbook,
    farm,
    states=states,
    rotor_model="centre",
    wake_models=["RHB_linear", "Bastankhah_linear"],
    wake_frame="rotor_wd",
    partial_wakes_model="auto",
    chunks={FC.STATE: None, FC.POINT: 4000},
    verbosity=1,
)

We can now calculate the results:

In [ ]:
with foxes.utils.runners.DaskRunner() as runner:
    farm_results = runner.run(algo.calc_farm)

print(farm_results)

Finally, we display the mean wind speed field as a 2D plot, including wake effects. We could simply do this by

```python
o = foxes.output.FlowPlots2D(algo, farm_results)
o.get_mean_fig_xy(FV.WS, resolution=30)
plt.show()
```

This is perfectly fine, but if we wish to see a progress bar and have additional options for the parallel run on a cluster (which we are not using here), the output function should instead be passed to a `DaskRunner` object:

In [ ]:
with foxes.utils.runners.DaskRunner() as runner:
    o = foxes.output.FlowPlots2D(algo, farm_results, runner=runner)
    o.get_mean_fig_xy(FV.WS, resolution=30)
    plt.show()

Note that for the vectorization/parallelization to work we need to set the `chunks` arguments in the creation of the `Algorithm` object. Here chunks of 4000 points were calculated at a time (for all states).